In [1]:
using NPZ
using PyPlot
using HDF5
using JLD
using LaTeXStrings
using QuanticsTCI
import TensorCrossInterpolation as TCI

rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
#rcParams["backend"] = :tk
rcParams["font.size"] = 9
rcParams["text.usetex"] = true
rcParams["font.family"] = "serif"
#rcParams["text.latex.preamble"] = [raw"\usepackage{amsmath}", raw"\usepackage{amssymb}"]
rcParams["font.serif"] = ["Computer Modern Roman"]
rcParams["lines.linewidth"] = 1.0
rcParams["lines.markersize"] = 2
rcParams["image.cmap"] = "coolwarm"

function maxabs(data)
    return maximum(abs.(data))
end

function load_Acont(filename)
    f = h5open(filename, "r")
    return read(f, "ωcont"), read(f, "Acont")
end

load_Acont (generic function with 1 method)

In [2]:
filename_3p = "/home/Anxiang.Ge/Desktop/PhD/code/MuNRG/MyMuNRG/data/plotdata_PSF_((F1,F1dag,Q34)).h5"
filename_4p = "/home/Anxiang.Ge/Desktop/PhD/code/MuNRG/MyMuNRG/data/plotdata_PSF_((F1,F1dag,F3,F3dag)).h5"

ωcont, PSF_3p = load_Acont(filename_3p);
ωcont, PSF_4p = load_Acont(filename_4p);


In [14]:
vmax = maximum(abs.(PSF_3p))
kwargs = Dict(:vmax=>vmax, :vmin=>-vmax)

Dict{Symbol, Float64} with 2 entries:
  :vmax => 1.01251
  :vmin => -1.01251

In [10]:
vmax = maximum(abs.(PSF_3p))
kwargs_3p = Dict(:vmax=>vmax, :vmin=>-vmax, :extent=>(250, 750, 250, 750))
vmax = maximum(abs.(PSF_4p))
kwargs_4p = Dict(:vmax=>vmax, :vmin=>-vmax, :extent=>(250, 750, 250, 750))

fig, axs = subplots(ncols=2, nrows=1)
axs[1].imshow(PSF_3p; kwargs_3p...) |> colorbar
axs[2].imshow(PSF_4p[:,:,512]; kwargs_4p...) |> colorbar
#axs[2, 1].imshow(real(interpdata_K3); kwargs...) |> colorbar
#axs[2, 2].imshow(imag(interpdata_K3); kwargs...) |> colorbar

plt.savefig("interpdata_PSFs_3p4p.png")

In [3]:
qR_K2

UndefVarError: UndefVarError: `qR_K2` not defined

In [3]:
### QTCI K2a: ###


R_K2 = 10
qR_K2 = 2^R_K2
qmesh = collect(Int, range(1, qR_K2, length=qR_K2))
println("extent of qmesh: \t [1, ", qR_K2, "]")

tol_high = 1e-3
tol_low  = 1e-4


qttK2a, _, _ = quanticscrossinterpolate(
    Float64,
    (x, y) -> PSF_3p[x, y],
    [qmesh, qmesh],
    tolerance=tol_high
)

qttK2a2, _, _ = quanticscrossinterpolate(
    Float64,
    (x, y) -> PSF_3p[x, y],
    [qmesh, qmesh],
    tolerance=tol_low
)


extent of qmesh: 	 [1, 1024]


(QuanticsTCI.QuanticsTensorCI2{Float64}(TensorCrossInterpolation.TensorCI2{Float64} with rank 42, QuanticsTCI.UnfoldingSchemes.interleaved), [2, 4, 8, 16, 20, 30, 32, 38, 41, 42, 43, 44, 47, 47, 42], [0.0, 0.0, 0.0, 4.9894903552796595e-5, 5.079266767209787e-5, 5.555997705579742e-5, 5.555997705579742e-5, 5.633977510976751e-5, 5.633977510976751e-5, 4.552396128546828e-5, 4.1186705961332024e-5, 5.2244334653028606e-5, 5.2244334653028606e-5, 5.224433465302105e-5, 4.1186705961332024e-5])

In [3]:
### QTCI K3a: ###

dict_K3 = Dict("R"=>10, "PSFdata"=>filename_4p)

R_K3 = 10#dict_K3["R"]
qR_K3 = 2^R_K3
qmesh = collect(Int, range(1, qR_K3, length=qR_K3));




In [4]:


tol_high = 1e-3
tol_low  = 1e-4

qttK3a, _, _ = quanticscrossinterpolate(
    ComplexF64,
    (x, y, z) -> PSF_4p[x, y, z],
    [qmesh, qmesh, qmesh],
    tolerance=tol_high
)

qttK3a2, _, _ = quanticscrossinterpolate(
    ComplexF64,
    (x, y, z) -> PSF_4p[x, y, z],
    [qmesh, qmesh, qmesh],
    tolerance=tol_low
)

(QuanticsTCI.QuanticsTensorCI2{ComplexF64}(TensorCrossInterpolation.TensorCI2{ComplexF64} with rank 181, QuanticsTCI.UnfoldingSchemes.interleaved), [2, 4, 8, 16, 32, 64, 93, 145, 159, 183, 185, 184, 181], [0.0, 1.9067948190860327e-5, 1.9067948190860327e-5, 2.035984802441095e-5, 5.2163460376526284e-5, 8.307058048350815e-5, 8.307058048350815e-5, 7.641749844241002e-5, 7.641749844241002e-5, 5.776580959408332e-5, 5.776580959408332e-5, 2.1722066745801726e-5, 3.616874064257676e-5])

In [6]:
PSF_4p = 0
GC.gc()

In [7]:
PSF_4p

0

In [14]:
function save_plotdata(qtt, Kclass_str, qttdata2D, dict_K)
    file = h5open("plotdata_PSFs.h5", "cw")
    g_K = create_group(file, Kclass_str)
    g_K["qttdata2D"] = qttdata2D
    g_K["qplotmesh"] = qplotmesh
    g_K["pivoterrors"] = qtt.tt.pivoterrors
    g_K["linkdims"] = TCI.linkdims(qtt.tt)
    save("qtt_dict_"*Kclass_str*".jld", "qtt_dict", dict_K)
    close(file)
    return nothing
end

save_plotdata (generic function with 1 method)

In [ ]:
zoom = 2^4
qplotmesh = Int.(round.(range(1, qR_K2, length=300)))
box = (-extent, extent, -extent, extent) ./ zoom
qttK2adata = qttK2a.(qplotmesh, qplotmesh')

zoom = 2^4
Length = 128
qplotmesh = Int.(round.(range(1, qR_K3, length=Length)))
box = (-extent, extent, -extent, extent) #./ zoom
qttK3adata = qttK3a.(qplotmesh, qplotmesh',2^(R_K3-1))




qR_K3 = 2^R_K3
qmesh = range(-extent, extent, length=qR_K3) |> collect
    


In [16]:
vmax = maximum(abs.(PSF_3p))


0

In [13]:
zoom = 2^4
qplotmesh = Int.(round.(range(1, qR_K2, length=512)))
vmax = maximum(abs.(PSF_3p))
kwargs = Dict(:vmax=>vmax, :vmin=>-vmax)
#box = (-ωlim, ωlim, -ωlim, ωlim)

qttK2adata = qttK2a.(qplotmesh, qplotmesh')
fig, axs = subplots(ncols=2, nrows=2, figsize=(300, 250)./72)

axs[1, 1].set_title(L"\mathrm{Re}(K_{2a})")
axs[1, 1].imshow(real(qttK2adata)'; kwargs...) |> colorbar
#axs[1, 2].set_title(L"\mathrm{Im}(K_{2a})")
#axs[1, 2].imshow(imag(qttK2adata)'; kwargs...) |> colorbar

axs[1, 1].set_ylabel(L"\nu")
for ax in axs[1, :]
    ax.set_xlabel(L"\omega")
end

axs[2, 1].semilogy(1:TCI.rank(qttK2a.tt), qttK2a.tt.pivoterrors, label=L"\epsilon=10^{-3}")
axs[2, 1].semilogy(1:TCI.rank(qttK2a2.tt), qttK2a2.tt.pivoterrors, label=L"\epsilon=10^{-4}")
axs[2, 1].set_xlabel(L"D_{\max}")
axs[2, 1].set_ylabel("abs. error")

axs[2, 2].semilogy(1:2R_K2-1, 2 .^ min.(1:2R_K2-1, 2R_K2-1:-1:1), color="gray", linewidth=0.5)
axs[2, 2].semilogy(1:2R_K2-1, TCI.linkdims(qttK2a.tt))
axs[2, 2].semilogy(1:2R_K2-1, TCI.linkdims(qttK2a2.tt))
axs[2, 2].set_xlabel(L"\ell")
axs[2, 2].set_ylabel(L"D_\ell")
axs[2, 2].set_ylim(1.5, 300)
axs[2, 2].set_xticks([1, 10, 19])

axs[2, 1].legend()
fig.suptitle("QTCI of 3p PSF")
tight_layout()


fig.savefig("PSF_3p.pdf")


In [20]:
maximum(abs.(PSF_4p))

In [19]:
Length = 128#256
qplotmesh = Int.(round.(range(250, 750, length=Length)))
#box = (-extent, extent, -extent, extent) #./ zoom

fig, axs = subplots(ncols=2, nrows=2, figsize=(300, 250)./72)
qttK3adata = qttK3a.(qplotmesh, qplotmesh',2^(R_K3-1))
vmax = maximum(abs.(qttK3adata))
kwargs = Dict(:vmax=>vmax, :vmin=>-vmax)

axs[1, 1].set_title(L"\mathrm{Re}(K_{3a;\, \omega,\nu,\nu'})")
axs[1, 1].imshow(real(qttK3adata)'; kwargs...) |> colorbar
#axs[1, 2].set_title(L"\mathrm{Im}(K_{3a})")
#axs[1, 2].imshow(imag(qttK3adata)'; extent=box, kwargs...) |> colorbar

axs[1, 1].set_ylabel(L"\nu'")
for ax in axs[1, :]
    ax.set_xlabel(L"\nu")
end
axs[2, 1].semilogy(1:TCI.rank(qttK3a.tt), qttK3a.tt.pivoterrors, label=L"\epsilon=10^{-3}")
axs[2, 1].semilogy(1:TCI.rank(qttK3a2.tt), qttK3a2.tt.pivoterrors, label=L"\epsilon=10^{-4}")
axs[2, 1].set_xlabel(L"D_{\max}")
axs[2, 1].set_ylabel("abs. error")

axs[2, 2].semilogy(1:3R_K3-1, 2 .^ min.(1:3R_K3-1, 3R_K3-1:-1:1), color="gray", linewidth=0.5)
axs[2, 2].semilogy(1:3R_K3-1, TCI.linkdims(qttK3a.tt))
axs[2, 2].semilogy(1:3R_K3-1, TCI.linkdims(qttK3a2.tt))
axs[2, 2].set_xlabel(L"\ell")
axs[2, 2].set_ylabel(L"D_\ell")
#axs[2, 2].set_ylim(1.5, 300)
axs[2, 2].set_xticks([1, 10, 19])

axs[2, 1].legend()
fig.suptitle("QTCI of 4p PSF")
tight_layout()


fig.savefig("PSF_4p.pdf")

In [18]:
#save_plotdata(qttK2a,  "u"*string(u_here)*"_K2a" *"_"*keldyshComponentsK2a*"_3", qttK2adata, Dict("R"=>R_K2, "extent"=>extent, "parquetdata"=>filename, "tol"=>tol_high))
#save_plotdata(qttK2a2, "u"*string(u_here)*"_K2a2"*"_"*keldyshComponentsK2a*"_3", qttK2adata, Dict("R"=>R_K2, "extent"=>extent, "parquetdata"=>filename, "tol"=>tol_low))
save_plotdata(qttK3a,  "4pPSF_1", qttK3adata, Dict("R"=>R_K3, "parquetdata"=>filename_4p, "tol"=>tol_high))
save_plotdata(qttK3a2, "4pPSF_2", qttK3adata, Dict("R"=>R_K3, "parquetdata"=>filename_4p, "tol"=>tol_low))